## Auteur
- [**Ferhat SAIDOUN**](https://github.com/ferhatte10)
- [**Zakaria RAJI**](https://github.com/RAJI-Zakaria)

# TP 2 : Canal de diffusion anonyme et génération de secret

# 1 - Canal de diffusion anonyme

Pour cette première partie, vous devez écrire une classe qui simule un ca-nal de diffusion anonyme (de type forum) et permet de réaliser les opérations suivantes :
1. Poster un message de manière anonyme. Votre fonction s’intitulant posterMessageAnonyme doit prendre en entrée une chaîne de caractères correspondant au message et le poster dans la structure qui servira le canal de diffusion anonyme. Libre à vous de choisir la struc-ture de données qui vous semble la plus appropriée pour implémenter le canal de diffusion anonyme. Le message posté doit aussi comporter une étiquette temporelle indiquant à quel moment le message a été diffusé (par exemple 16h39m25s). La structure de données que vous utilisez doit ordonner les messages chronologiquement en se basant sur cette étiquette temporelle.
2. Lire une série de messages postées de manière anonyme. Votre fonction s’intitulant recupererMessagesAnonymes prendra comme arguments d’entrée deux étiquettes temporelles (une de début et une defin de période) et retournera la liste de tous les messages anonymes postés durant cette période.

In [ ]:
import datetime
import time
import random
import threading

class CanalDiffusionAnonyme: # Classe pour simuler un canal de diffusion anonyme
    def __init__(self): # Constructeur de la classe
        self.messages = [] # Liste pour stocker les messages
        self.lock = threading.Lock() # Verrou pour éviter les problèmes de concurrence

    def posterMessageAnonyme(self, message): # Fonction pour poster un message de manière anonyme
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %Hh%Mm%Ss") # Étiquette temporelle
        new_message = {"message": message, "timestamp": timestamp} # Création du message
        with self.lock: # pour éviter les problèmes de concurrence
            self.messages.append(new_message) # Ajout du message à la liste
            self.messages = sorted(self.messages, key=lambda x: x["timestamp"])  # trier par étiquette temporelle

    def recupererMessagesAnonymes(self, debut, fin): # Fonction pour récupérer les messages postés de manière anonyme entre deux étiquettes temporelles
        return [msg for msg in self.messages
                                if debut <= msg["timestamp"] <= fin] # Retourne les messages entre les deux étiquettes temporelles

## 2 - Génération de secret via canal de diffusion ano-nyme

À partir simplement de la primitive du canal de diffusion anonyme, ilest possible de réaliser une génération de secret (à partir de rien) entre deuxentités tel que même si un adversaire passif prend lui aussi connaissance detous les messages il n’apprend aucune information sur le secret. Pour finsd’illustration, on va supposer que les deux entités se nomment Alice et Bob.Ils vont réaliser le protocole de génération de secret suivant :
1. Alice tire aléatoirement à pile ou face un bitb
2. Si le bitbvaut 0, elle construit un message comprenant le nom "Alice"et sinon pourb= 1elle construit un message comprenant le nom"Bob".
3. Alice poste ce message de manière anonyme après un temps aléatoirepris au hasard dans une certaine fourchette (par exemple entre 1 et10ms)
4. Alice recommence à partir de l’étape 1.

Du côté de Bob il réalise en parallèle le même protocole à la différence prèsque ses réactions sont inversées, sib= 0il construit un message comprenantle nom "Bob" et sinon sib= 1son message sera "Alice"

On suppose qu’Alice et Bob gardent chacun une liste de messages qu’ils ont envoyées (avec les étiquettes temporelles correspondantes). Le protocolese déroule pendant une certaine période (par exemple 1 minute) dont le moment de début et la durée ont pu être négociées par Alice et Bob aupréalable par exemple en utilisant le canal de diffusion anonyme.
Après que cette période se soit écoulée, Alice et Bob extraient le secret (qui correspondra à une chaîne de bits) de la manière suivante. Ils regardent les messages dans l’ordre chronologique (on suppose par exemple quele nombre de messages est n). Pour chaque message d’indexisi le messagecontient le texte "Alice" et qu’il a vraiment été envoyé par Alice alorssi,le bit secret généré, vaudra 0 (même chose si le message contient le texte"Bob" et qu’il a vraiment été envoyé par Bob). À l’opposé si le messagecontient le texte "Alice" mais qu’il a été envoyé par Bob alorssivaudra 1(même chose si chose si le message contient le texte "Bob" mais qu’il a étéenvoyé par Alice). Une fois être passé à travers les n messages Alice et Bob se retrouvent en possession d’un secrets sous la forme d’une chaîne aléatoire de n bits.

Pour implémenter cette deuxième partie, vous devez écrire deux fonctionsqui réalisent les opérations suivantes :
1. Génération de secret en utilisant le canal de diffusion anonyme. Votre fonction s’intitulant generer Secret doit simuler le protocole de généra-tion de secret tel que décrit plus haut en utilisant le canal de diffusion anonyme que vous avez implémenté dans la partie 1. Cette fonctionprendre en entrée au moins trois paramètres, deux chaînes de carac-tère représentant le nom des interlocuteurs (par exemple "Alice" et"Bob" dans l’exemple ci-dessus) ainsi que la durée pendant laquelle ils doivent réaliser ce protocole.



In [ ]:
# 1. Génération de secret en utilisant le canal de diffusion anonyme

def genererSecret(canalAnonyme, nom_Alice, nom_Bob, duree): # Fonction pour générer un secret en utilisant le canal de diffusion anonyme
    debut = datetime.datetime.now() # Début du protocole
    fin = debut + datetime.timedelta(seconds=duree) # Fin du protocole

    vues_Alice = []  # Liste pour stocker les vues d'Alice
    vues_Bob = []  # Liste pour stocker les vues de Bob

    def process_person(person_sender, person_dist, views_Alice_list, views_Bob_list): # Fonction pour simuler le protocole de génération de secret
        while datetime.datetime.now() < fin: # Tant que le protocole n'est pas terminé
            bit_person = random.randint(0, 1) # Tire aléatoirement un bit (0 ou 1)
            if bit_person == 0: # Si le bit est 0
                message_person = f"{person_sender}" # Le message contient le nom de l'expéditeur
            else: # Si le bit est 1
                message_person = f"{person_dist}" # Le message contient le nom du destinataire
            time.sleep(random.uniform(0.001, 0.01)) # Attend un temps aléatoire entre 1 et 10 ms
            canalAnonyme.posterMessageAnonyme(message_person) # Poste le message de manière anonyme

            # Ajouter la vue correspondante pour chaque participant
            if person_sender == nom_Alice: # Si l'expéditeur est Alice
                views_Alice_list.append(True)  # Ajoute True à la vue d'Alice
                views_Bob_list.append(False)  # Ajoute False à la vue de Bob
            else: # Si l'expéditeur est Bob
                views_Alice_list.append(False)  # Ajoute False à la vue d'Alice
                views_Bob_list.append(True)  # Ajoute True à la vue de Bob

    thread_alice = threading.Thread(target=process_person, args=(nom_Alice, nom_Bob, vues_Alice, vues_Bob)) # Création d'un thread pour Alice
    thread_bob = threading.Thread(target=process_person, args=(nom_Bob, nom_Alice, vues_Alice, vues_Bob)) # Création d'un thread pour Bob

    thread_alice.start() # Démarrage du thread d'Alice
    thread_bob.start() # Démarrage du thread de Bob

    thread_alice.join() # Attente de la fin du thread d'Alice
    thread_bob.join() # Attente de la fin du thread de Bob

    # Récupérer les messages postés pendant la durée du protocole
    messages_postes = canalAnonyme.recupererMessagesAnonymes(debut.strftime("%Y-%m-%d %Hh%Mm%Ss"), fin.strftime("%Y-%m-%d %Hh%Mm%Ss")) # Récupère les messages postés pendant la durée du protocole (entre le début et la fin)

    return messages_postes, vues_Alice, vues_Bob  # Retourne les messages postés, les messages générés, les vues d'Alice et les vues de Bob  

2. Extraction de secret. Votre fonction s’intitulant extraireSecret doit permettre à partir d’un transcript de messages obtenu suite à l’application de genererSecret combiné avec la vue partielle des deux entités d’extraire un secret commun. Le secret sera de taille n, le nombre de messages du transcript.

Il vous est aussi demandé d’écrire un court programme qui implémente un scénario servant à tester les différentes fonctions que vous aurez écrit durant ce TP.

In [60]:
# 2. Extraction de secret à partir d'un transcript de messages
def extraireSecret(transcript_messages, views_Alice, views_Bob): # Fonction pour extraire le secret à partir d'un transcript de messages
    secret = ""  # Initialise le secret comme une chaîne vide

    if len(transcript_messages) != len(views_Alice) or len(transcript_messages) != len(views_Bob): # Vérifie si les longueurs des vues correspondent à la longueur du transcript
        print("Erreur: Les longueurs des vues ne correspondent pas à la longueur du transcript")
        return None  # Sortie de la fonction en cas d'erreur de taille

    # Parcourt les messages, la vue d'Alice et la vue de Bob
    for i in range(len(transcript_messages)): # Pour chaque message
        message = transcript_messages[i]["message"] # Récupère le message
        alice_view = views_Alice[i] # Récupère la vue d'Alice
        bob_view = views_Bob[i] # Récupère la vue de Bob
        # Vérifie si le message correspond à la vue d'Alice et de Bob
        if alice_view and bob_view:  # Si les deux vues sont vraies
            print("Erreur: Le message ne peut pas correspondre aux deux vues")
            return None  # Sortie de la fonction en cas d'erreur de vue
        elif alice_view and not bob_view:  # Vue d'Alice uniquement
            if "Alice" in message:
                secret += "0"
            elif "Bob" in message:
                secret += "1"
        elif bob_view and not alice_view:  # Vue de Bob uniquement
            if "Alice" in message:
                secret += "1"
            elif "Bob" in message:
                secret += "0"
        else:
            # Gère les autres cas si nécessaire
            pass
    return secret  # Retourne le secret extrait

# Programme principal
La fonction main ci-dessous permet de tester les fonctions précédentes.

In [67]:
def main(): # Exemple d'utilisation 
    canal = CanalDiffusionAnonyme() # Création d'un canal de diffusion anonyme
    nom1 = "Alice" # Nom de la première entité
    nom2 = "Bob" # Nom de la deuxième entité
    duree_protocole = 1 # Durée du protocole en secondes
    
    messages,view_Alice, view_Bob = genererSecret(canal, nom1, nom2, duree_protocole) # Génération du secret
    
    secret = extraireSecret(messages, view_Alice, view_Bob) # Extraction du secret
    
    # Affichage des résultats obtenus sur les 5 premiers messages pour vérifier le bon fonctionnement du protocole
    print("Messages postés:", list(map(lambda x: x["message"], messages))[:5])
    print("Vues d'Alice:", view_Alice[:5]) # Affichage des vues d'Alice
    print("Vues de Bob:", view_Bob[:5]) # Affichage des vues de Bob
    print("Secret extrait:", secret[:5]) # Affichage du secret extrait
    
if __name__ == "__main__":
    main()

Messages postés: ['Alice', 'Bob', 'Bob', 'Bob', 'Alice']
Vues d'Alice: [True, False, False, True, True]
Vues de Bob: [False, True, True, False, False]
Secret extrait: 00010


# Rapport : Applications et Sécurité des Canaux de Diffusion Anonyme

## Applications du Canal de Diffusion Anonyme

Le canal de diffusion anonyme est une structure cruciale avec de multiples applications au-delà de la simple diffusion de messages. Deux autres utilisations possibles sont :

- **Votations et Sondages Anonymes :** Le canal de diffusion anonyme pourrait servir à organiser des votations électorales ou des sondages en garantissant l'anonymat des votants. Les participants pourraient soumettre leurs votes ou opinions sans révéler leur identité, garantissant ainsi l'intégrité du processus.

- **Plateforme de Signalement Anonyme :** Il pourrait être utilisé comme une plateforme sécurisée pour les dénonciations anonymes. Les utilisateurs pourraient signaler des problèmes de sécurité, des comportements inappropriés ou des incidents sans craindre de représailles, assurant la confidentialité et la sécurité des informateurs.

## Sécurité du Protocole entre Alice et Bob

Le protocole de génération de secret décrit dans la deuxième partie permet à Alice et Bob de créer un secret commun sans révéler d'informations exploitables à un adversaire passif. Ceci est dû à plusieurs raisons :

- **Anonymat des Messages :** Les messages sont postés de manière aléatoire et anonyme par Alice et Bob. Même si l'adversaire surveille les messages, il ne peut pas associer de manière fiable les messages à un expéditeur spécifique.

- **Impossibilité de Corrélation :** Le secret est généré en utilisant des messages anonymes et des vues distinctes. Les vues ne permettent pas à l'adversaire de corréler les actions d'Alice et de Bob, rendant impossible la déduction du secret à partir des seuls messages observés.

- **Indépendance des Vues :** Les vues d'Alice et de Bob sont indépendantes et ne fournissent pas d'informations sur les actions de l'autre partie. Même si l'adversaire a une vue sur les messages, il ne peut pas reconstituer le secret sans les vues des deux entités.

## Importance des Primitives de Génération de Secret Commun

Avoir des primitives permettant à deux entités de générer un secret commun à travers un canal potentiellement surveillé est crucial pour la sécurité des communications sensibles. Cela garantit que même si le canal de communication est compromis ou surveillé par un adversaire, les entités légitimes peuvent toujours créer et partager des secrets sans compromettre leur confidentialité et leur sécurité.
